<a href="https://colab.research.google.com/github/princessivy/Analyzing_CSGO_SkinPrices/blob/main/CSGO_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import pandas as pd
import requests
import json

from google.colab import drive
from pathlib import Path

from PIL import Image
import requests
import cv2
import numpy as np

#Daten einlesen

In [2]:
data = pd.read_csv(r'/content/drive/MyDrive/SA_BA/Data/allitems.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#Beschreibung der Features
Der rohe Datensatz enthält die folgenden Features:

In [3]:
data.columns

Index(['id', 'saleId', 'productId', 'assetId', 'itemId', 'appid', 'steamid',
       'url', 'family', 'family_localized', 'name', 'title', 'text',
       'marketName', 'marketHashName', 'color', 'bgColor', 'image', 'classid',
       'assetid', 'lock', 'version', 'versionType', 'stackAble',
       'suggestedPrice', 'salePrice', 'currency', 'saleStatus', 'saleType',
       'category', 'category_localized', 'subCategory',
       'subCategory_localized', 'pattern', 'finish', 'customName', 'wear',
       'link', 'type', 'exterior', 'quality', 'rarity', 'rarity_localized',
       'rarityColor', 'collection', 'collection_localized', 'stickers',
       'canHaveScreenshots', 'screenshots', 'souvenir', 'stattrak', 'tags',
       'ownItem'],
      dtype='object')

Folgende Tabelle erklärt die Bedeutung der einzelnen Features:

Feature | Bedeutung
--- | ---
id | wurde nicht herausgefunden, ist stetig '0'
saleId | eindeutige Verkaufs-ID
productId | eindeutige Produkt-ID
assetId | eindeutige Asset-ID
itemId| itemspezifische ID
appid | Zuordnung zur Applikation CS:GO, ist stetig '730'
steamid | KP??
url | spezifischer Bestandteil der gesamten URL für das Item
family | zugehörige Familie des Items (Unterunterkategorie)
family_localized | wie 'family' nur übersetzt in Sprache der Abfrage (hier: weiterhin englisch)
name | Anzeigename auf Skinport
title | angezeigte Unterkategorie zu jedem Item
text | kurze Beschreibung zum Item
marketName | Zusammensetzung aus 'title' und 'name'
marketHashName | wurde nicht herausgefunden, ähnlich zu 'marketName'
color | 4 Farben?! :D
bgColor | Wert nicht gepflegt, durchgehend 'NaN'
image | mit Ergänzung "https://steamcommunity-a.akamaihd.net/economy/image/" vor dem Wert, gelangt man an ein standardisiertes Cover-Bild des Items
classid | wurde nicht herausgefunden
assetid | weitere eindeutige Asset-ID
lock | Zeigt, ab wann Item zum Spieleinsatz freigegeben ist ('NaN' bedeutet, dass ein direkter Einsatz möglich ist)
version | verschiedene Doppler-Phasen von CS:GO
versionType | Zuordnung der 'version's in übergeordnete Kategorie
stackAble | wurde nicht herausgefunden, ist stetig 'False'
suggestedPrice | Wertvorschlag des Preises von Skinport
salePrice | Kaufpreis des Items
currency | Währung, ist stetig 'EUR'
saleStatus | gibt an, ob Item verkaufbar ist; ist stetig 'listed' und somit verkaufbar
saleType | gibt an, wer einen Kauf machen kann; ist stetig 'public' und somit für alle zugänglich
category | Überkategorie
category_localized | wie 'category' nur übersetzt in Sprache der Abfrage (hier: weiterhin englisch)
subCategory | Unterkategorie
subCategory_localized | wie 'subCategory' nur übersetzt in Sprache der Abfrage (hier: weiterhin englisch)
pattern | Musternummer des Items (zwischen 1 und 1000)
finish | Finish des Items
customName | Name des Items, von einem Besitzer hinzugefügt
wear | Abnutzung (zwischen 0 und 1)
link | wurde nicht herausgefunden
type | Typ des Items
exterior | Kategorie der Abnutzung, je nach 'wear'
quality | Kategorie der Qualität
rarity | Kategorie der Rarität
rarity_localized | wie 'rarity' nur übersetzt in Sprache der Abfrage (hier: weiterhin englisch)
rarityColor | Farbkategorie der Rarität
collection | Kollektion 
collection_localized | wie 'collection' nur übersetzt in Sprache der Abfrage (hier: weiterhin englisch)
stickers | Array mit Stickern, welche sich auf dem Item befinden
canHaveScreenshots | gibt an, ob Screenshots zum Item existieren; ist stetig 'True'
screenshots | gibt an, welche Screenshots zum Item existieren
souvenir | gibt an, ob das Item ein Souvenir ist oder nicht
stattrak | gibt an, ob das Item Stattrak-Eigenschaft hat oder nicht
tags | angegebene Tags auf Skinport
ownItem | gibt an, ob das Item unter Besitz steht; ist stetig 'False'



Nun werden einige Features entfernt, die unbrauchbar sind. Nähere Erläuterung in Kapitel X der Seminararbeit.
(wenn immer nur gleicher Wert, können die auch weg, weil Varianz gleich;; wenn alle unterschiedlich sind, bringt das auch nix)

In [6]:
data = data.drop(['id', 'saleId', 'productId', 'itemId', 'appid', 'steamid', 'url', 'family_localized', 'name', 'title', 'text', 'marketName', 'marketHashName', 'bgColor',
                  'image', 'classid', 'assetid', 'stackAble', 'currency', 'saleStatus', 'saleType', 'category_localized', 'subCategory_localized',
                  'customName', 'link', 'rarity_localized', 'collection_localized', 'canHaveScreenshots', 'screenshots', 'tags', 'ownItem'], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
data.columns

Index(['assetId', 'family', 'color', 'lock', 'version', 'versionType',
       'suggestedPrice', 'salePrice', 'category', 'subCategory', 'pattern',
       'finish', 'wear', 'type', 'exterior', 'quality', 'rarity',
       'rarityColor', 'collection', 'stickers', 'souvenir', 'stattrak'],
      dtype='object')

In [8]:
len(data.columns)

22

#Explorative Datenanalyse
Eindruck über die Daten:

In [20]:
data.head()

,assetId,family,color,lock,version,versionType,suggestedPrice,salePrice,category,subCategory,...,wear,type,exterior,quality,rarity,rarityColor,collection,stickers,souvenir,stattrak
0,46917102,Tiger Tooth,#8650AC,NaN,default,default,39721,39913,Knife,Bayonet,...,0.033243,★ Covert Knife,Factory New,★,Covert,#eb4b4b,NaN,[],False,False
1,60753316,Tiger Tooth,#8650AC,2022-04-02T07:00:00.000Z,default,default,44203,36244,Knife,Bayonet,...,0.025035,★ StatTrak™ Covert Knife,Factory New,★ StatTrak™,Covert,#eb4b4b,NaN,[],False,True
2,55716802,Tiger Tooth,#8650AC,NaN,default,default,39721,40000,Knife,Bayonet,...,0.006977,★ Covert Knife,Factory New,★,Covert,#eb4b4b,NaN,[],False,False
3,43496004,Tiger Tooth,#8650AC,NaN,default,default,39721,42073,Knife,Bayonet,...,0.006662,★ Covert Knife,Factory New,★,Covert,#eb4b4b,NaN,[],False,False
4,60829717,Tiger Tooth,#8650AC,2022-04-03T07:00:00.000Z,default,default,39721,35500,Knife,Bayonet,...,0.062841,★ Covert Knife,Factory New,★,Covert,#eb4b4b,NaN,[],False,False


Datentypen angucken. Wichtige Unterscheidung zwischen numerischen und kategorialen Features:

In [37]:
data.dtypes

assetId             int64
family             object
color              object
lock               object
version            object
versionType        object
suggestedPrice      int64
salePrice           int64
category           object
subCategory        object
pattern           float64
finish            float64
wear              float64
type               object
exterior           object
quality            object
rarity             object
rarityColor        object
collection         object
stickers           object
souvenir             bool
stattrak             bool
dtype: object

Numerisch:
- assetId
- suggestedPrice
- salePrice
- pattern
- finish
- wear

Kategorisch:
- family
- color
- lock
- version
- versionType
- category
- subCategory
- type
- exterior
- quality
- rarity
- rarityColor
- collection
- stickers
- souvenir
- stattrak


####Missing-Values

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133005 entries, 0 to 133004
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   assetId         133005 non-null  int64  
 1   family          133005 non-null  object 
 2   color           133005 non-null  object 
 3   lock            29404 non-null   object 
 4   version         133005 non-null  object 
 5   versionType     133005 non-null  object 
 6   suggestedPrice  133005 non-null  float64
 7   salePrice       133005 non-null  float64
 8   category        133005 non-null  object 
 9   subCategory     133005 non-null  object 
 10  pattern         132989 non-null  float64
 11  finish          132838 non-null  float64
 12  wear            133005 non-null  float64
 13  type            133005 non-null  object 
 14  exterior        133005 non-null  object 
 15  quality         133005 non-null  object 
 16  rarity          133005 non-null  object 
 17  rarityColo

In [90]:
data.shape

(133005, 22)

In [91]:
data = data.drop_duplicates()

In [92]:
data.shape #es gibt keine Duplikate

(133005, 22)

In [93]:
data.isnull().sum()

assetId                0
family                 0
color                  0
lock              103601
version                0
versionType            0
suggestedPrice         0
salePrice              0
category               0
subCategory            0
pattern               16
finish               167
wear                   0
type                   0
exterior               0
quality                0
rarity                 0
rarityColor            0
collection          4680
stickers               0
souvenir               0
stattrak               0
dtype: int64

In den Features *lock, pattern, finish* und *collection* finden sich Null-Werte. Diese werden nun behandelt. 

**lock** Bei *lock* geht es darum, ob die Items nach dem Kauf direkt zum Spiel einsetzbar sind: Ja oder Nein. Ein 'NaN' steht in diesem Fall für "ja"(1), eine Zeitangabe für "nein"(0). Entsprechend entsteht hieraus ein Boolean.

In [124]:
for x in range(len(data)):
  try:
    if np.isnan(data.lock[x]):
      data.lock[x] = 0
  except:
    data.lock[x] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


**pattern** Die fehlenden Werte lassen sich hier nicht einfach durch Binärisierung oder Imputation lösen. Solche Annahmen über das Pattern würden die Daten eher verfälschen. Da es sich bei den 16 fehlenden Werten um einen Anteil von 0,012% handelt, werden diese hiermit aus dem Datensatz entfernt.

In [138]:
data.shape

(133005, 22)

In [139]:
data = data[data['pattern'].notna()]

In [140]:
data.shape

(132989, 22)

**finish** Selbes Vorgehen wie bei *pattern*. Der Anteil, gemessen an der neuen Datenbasis, liegt bei 0,126%.

In [141]:
data = data[data['finish'].notna()]

In [142]:
data.shape

(132822, 22)

In [184]:
#Index wieder richten
data = data.reset_index()

**collection** Die Items, die keiner Kollektion zugeordnet sind, werden zur Kollektion "No Collection" umbenannt und bilden somit eine weitere Kategorie.

In [170]:
 data['collection'] = data['collection'].fillna("No Collection")

####Datentypenanpassung
*suggestedPrice* und *salePrice* sollten float's sein. Die Trennung zwischen Euros und Cents fehlt. Diese Variablen werden durch 100 geteilt, um dies zu lösen

In [80]:
data['salePrice'] = data['salePrice']/100
data['suggestedPrice'] = data['suggestedPrice']/100

*pattern* liegt zwischen 1 und 1000. Dabei handelt es sich nicht um einen float-Wert. Dieser Wert wird auf int gecastet.

In [175]:
data = data.astype({"pattern": int})

*finish* ist ebenso ein float, welcher eigentlich einem int enstpricht. Deshalb Casting.

In [197]:
data = data.astype({"finish": int})

Um das Feature *stickers* richtig zu verarbeiten, wird dieses zu einer boolschen Variable umgeschrieben. Sticker: Ja(1) oder Nein(0).

In [227]:
for x in range(len(data)):
  if data.stickers[x] == '[]':
    data.stickers[x] = 0
  else:
    data.stickers[x] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


*souvenir* steht aktuell auf "True" oder "False". Hier wird eine Umformung in 0 und 1 vorgenommen. Das selbe gilt für *stattrak*.

In [238]:
#souvenir
data["souvenir"] = data["souvenir"].astype(int)

#stattrak
data["stattrak"] = data["stattrak"].astype(int)

####Aktuelle Übersicht der Daten

In [243]:
data.describe()

,index,assetId,suggestedPrice,salePrice,pattern,finish,wear,souvenir,stattrak
count,132822.000000,1.328220e+05,132822.000000,132822.000000,132822.000000,132822.000000,132822.000000,132822.000000,132822.000000
mean,66580.896501,4.724622e+07,29.443286,34.422234,500.392533,647.262283,0.255292,0.063988,0.099991
std,38352.754248,1.329716e+07,202.335509,372.976511,288.729975,1106.288792,0.177134,0.244732,0.299989
min,0.000000,1.240700e+04,0.030000,0.020000,0.000000,2.000000,0.000007,0.000000,0.000000
25%,33377.250000,3.878514e+07,0.030000,0.030000,250.000000,136.000000,0.125445,0.000000,0.000000
50%,66583.500000,5.176906e+07,0.100000,0.120000,500.000000,611.000000,0.223606,0.000000,0.000000
75%,99792.750000,5.838940e+07,1.590000,3.000000,751.000000,841.000000,0.361484,0.000000,0.000000
max,133004.000000,6.096650e+07,16609.350000,49999.990000,1000.000000,10088.000000,0.999827,1.000000,1.000000


In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132822 entries, 0 to 132821
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   index           132822 non-null  int64  
 1   assetId         132822 non-null  int64  
 2   family          132822 non-null  object 
 3   color           132822 non-null  object 
 4   lock            132822 non-null  object 
 5   version         132822 non-null  object 
 6   versionType     132822 non-null  object 
 7   suggestedPrice  132822 non-null  float64
 8   salePrice       132822 non-null  float64
 9   category        132822 non-null  object 
 10  subCategory     132822 non-null  object 
 11  pattern         132822 non-null  int64  
 12  finish          132822 non-null  int64  
 13  wear            132822 non-null  float64
 14  type            132822 non-null  object 
 15  exterior        132822 non-null  object 
 16  quality         132822 non-null  object 
 17  rarity    

In [242]:
data.corr()

,index,assetId,suggestedPrice,salePrice,pattern,finish,wear,souvenir,stattrak
index,1.000000,-0.140313,-0.108998,-0.062490,0.000838,0.166137,0.083231,0.105433,-0.083764
assetId,-0.140313,1.000000,0.068147,0.030457,0.002101,0.148028,-0.134380,0.049078,0.145613
suggestedPrice,-0.108998,0.068147,1.000000,0.600735,-0.000626,0.139426,-0.093623,-0.029624,0.030637
salePrice,-0.062490,0.030457,0.600735,1.000000,0.000718,0.075264,-0.059444,-0.013751,0.027065
pattern,0.000838,0.002101,-0.000626,0.000718,1.000000,-0.000575,-0.000641,0.002937,-0.001540
finish,0.166137,0.148028,0.139426,0.075264,-0.000575,1.000000,-0.007651,0.017879,0.009869
wear,0.083231,-0.134380,-0.093623,-0.059444,-0.000641,-0.007651,1.000000,-0.079244,-0.020883
souvenir,0.105433,0.049078,-0.029624,-0.013751,0.002937,0.017879,-0.079244,1.000000,-0.087150
stattrak,-0.083764,0.145613,0.030637,0.027065,-0.001540,0.009869,-0.020883,-0.087150,1.000000


###Visualisierungen

In [ ]:
xx

###Strategie Datenaufbereitung für Modelle:


- assetId: passt
- family: eig zu viel
- color: Datentyp ändern, auf int64 umstellen, 4 Ausprägungen
- lock: passt
- version: genauer angucken, villeicht one hot encoding?
- versionType: kann man eig rausschmeißen, ist überordnung von version
- suggestedPrice: passt
- salePrice: passt
- Category: One Hot Encoding
- subCategory: viele ausprägungen... ohe?
- pattern: passt
- finish: passt
- wear: passt
- type: gute allg. Beschreibung, vollständig.. ohe?
- exterior: lable encoding
- quality: lable encoding
- rarity: lable encoding
- collection: nochmal angucken, ohe?
- stickers: passt
- souvenir: passt
- stattrack: passt

wenn zu viel OHE, dann frequency-encoding probieren?
https://medium.com/swlh/stop-one-hot-encoding-your-categorical-features-avoid-curse-of-dimensionality-16743c32cea4
"The frequency distribution of the column Country can be generated using the pandas' function: data[‘country’].value_counts()
Now replace each of the categories with their frequency in the data, for example, United States will be replaced by 7768, Russia by 1161. This technique is very each to implement and is a tremendously popular technique amongst Kagglers.
The downside of using this technique, if some categories have the same count, then they will be treated similarly by the model, hence losing some information."

####Daten in richriges Format bringen

numpy.float64

In [23]:
data.shape

(133005, 22)

In [24]:
data.describe()

,assetId,suggestedPrice,salePrice,pattern,finish,wear
count,1.330050e+05,1.330050e+05,1.330050e+05,132989.000000,132838.000000,133005.000000
mean,4.725580e+07,2.980865e+03,3.474846e+03,500.420666,647.236386,0.255393
std,1.329796e+07,2.026899e+04,3.729475e+04,288.741671,1106.231091,0.177147
min,1.240700e+04,3.000000e+00,2.000000e+00,0.000000,2.000000,0.000007
25%,3.879762e+07,3.000000e+00,3.000000e+00,250.000000,136.000000,0.125489
50%,5.179799e+07,1.000000e+01,1.200000e+01,500.000000,611.000000,0.223698
75%,5.840813e+07,1.620000e+02,3.050000e+02,751.000000,841.000000,0.361605
max,6.096650e+07,1.660935e+06,4.999999e+06,1000.000000,10088.000000,0.999827


In [25]:
data.category.describe()

count     133005
unique         6
top        Rifle
freq       43667
Name: category, dtype: object

In [26]:
data.category.value_counts()

Rifle     43667
Pistol    37351
SMG       28135
Heavy     17467
Knife      4771
Gloves     1614
Name: category, dtype: int64

In [27]:
data.corr()
'''Die Untersuchung bivariater Beziehungen ist nützlich, wenn Sie ein Ergebnis oder eine abhängige 
Variable im Auge haben und die Merkmale sehen möchten, die am stärksten mit der Zunahme oder Abnahme 
des Ergebnisses korrelieren. Sie können bivariate Beziehungen mit Hilfe von Streudiagrammen visuell 
darstellen (siehe unten im Abschnitt über die Darstellung).'''

,assetId,suggestedPrice,salePrice,pattern,finish,wear,souvenir,stattrak
assetId,1.000000,0.068925,0.030860,0.002147,0.148031,-0.133814,0.048783,0.145395
suggestedPrice,0.068925,1.000000,0.601326,-0.000339,0.139428,-0.092672,-0.030024,0.031478
salePrice,0.030860,0.601326,1.000000,0.000858,0.075266,-0.059053,-0.013972,0.027527
pattern,0.002147,-0.000339,0.000858,1.000000,-0.000575,-0.000563,0.002910,-0.001460
finish,0.148031,0.139428,0.075266,-0.000575,1.000000,-0.007651,0.017888,0.009874
wear,-0.133814,-0.092672,-0.059053,-0.000563,-0.007651,1.000000,-0.079354,-0.021019
souvenir,0.048783,-0.030024,-0.013972,0.002910,0.017888,-0.079354,1.000000,-0.087119
stattrak,0.145395,0.031478,0.027527,-0.001460,0.009874,-0.021019,-0.087119,1.000000


In [ ]:
data = data.drop_duplicates()

In [28]:
data.isnull()

,assetId,family,color,lock,version,versionType,suggestedPrice,salePrice,category,subCategory,...,wear,type,exterior,quality,rarity,rarityColor,collection,stickers,souvenir,stattrak
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133000,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
133001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
133002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
133003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [29]:
data.isnull().sum()

assetId                0
family                 0
color                  0
lock              103601
version                0
versionType            0
suggestedPrice         0
salePrice              0
category               0
subCategory            0
pattern               16
finish               167
wear                   0
type                   0
exterior               0
quality                0
rarity                 0
rarityColor            0
collection          4680
stickers               0
souvenir               0
stattrak               0
dtype: int64

In [ ]:
data = data.dropna()

In [30]:
data.std()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


assetId           1.329796e+07
suggestedPrice    2.026899e+04
salePrice         3.729475e+04
pattern           2.887417e+02
finish            1.106231e+03
wear              1.771472e-01
souvenir          2.445888e-01
stattrak          3.000663e-01
dtype: float64

In [208]:
data.groupby('category')['salePrice'].median()

category
Gloves    161.915
Heavy       0.040
Knife     235.000
Pistol      0.220
Rifle       0.350
SMG         0.040
Name: salePrice, dtype: float64

#Ausprobiertes

In [124]:
data.assetId[0]

46917102

In [17]:
data.subCategory[0]

'Bayonet'

In [18]:
data.family[0]

'Tiger Tooth'

In [24]:
data.type[0]

'★ Covert Knife'

In [33]:
data.stickers[11300]

"[{'sticker_id': None, 'wear': None, 'img': 'https://steamcdn-a.akamaihd.net/apps/730/icons/econ/stickers/community02/thuglife.a74c2a8f127182e257ff19650bb087aeee954bb3.png', 'name': 'Thug Life', 'type': None, 'slot': 1, 'color': None, 'name_localized': 'Thug Life', 'type_localized': None}, {'sticker_id': None, 'wear': None, 'img': 'https://steamcdn-a.akamaihd.net/apps/730/icons/econ/stickers/community02/thuglife.a74c2a8f127182e257ff19650bb087aeee954bb3.png', 'name': 'Thug Life', 'type': None, 'slot': 3, 'color': None, 'name_localized': 'Thug Life', 'type_localized': None}]"

In [37]:
data.stattrak[11300]

True

In [9]:
url = "https://cdn.skinport.com/images/screenshots/46917102/playside_256x128.png"

In [11]:
im = Image.open(requests.get(url, stream=True).raw)

In [12]:
rgb_image = im.convert('RGB')

In [15]:
pix = np.array(rgb_image)

In [16]:
pix.shape

(128, 256, 3)

In [17]:
def image_colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
	rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)

In [18]:
result = image_colorfulness(pix)

In [19]:
result

59.79024317069133